Use this notebook to split a video into separate frames and annotate the hand bounding boxes.

In [1]:
import vctoolkit as vc
import os
import predict_images_wo_annot as original_wrapper

In [2]:
# You need to set the following variables manually.
# input video
video_path = 'test/heart.mov'
# a folder that stores the video frames
frames_dir = 'test/frames'
# path to save the bounding box results
bbox_save_path = 'test/bbox.pkl'

Step 1: turn video into frames. You can skip this if you already have a folder of frames.

In [3]:
os.makedirs(frames_dir, exist_ok=True)
video = vc.VideoReader(video_path)
print(f'{video.n_frames} frames to process...')
for frame_idx in vc.progress_bar(range(video.n_frames), 'video frames'):
  frame = video.next_frame()
  if frame is None:
    break
  vc.save(os.path.join(frames_dir, f'{frame_idx:06d}.jpg'), frame)

video frames:   0%|          | 0/382 [00:00<?, ?it/s]

382 frames to process...


video frames: 100%|#########9| 381/382 [00:11<00:00, 33.48it/s]

Step 2: get file list to be processed.

In [4]:
file_list = []
for file_name in os.listdir(frames_dir):
    if os.path.splitext(file_name)[1][1:].lower() in ['jpg', 'png', 'jpeg']:
        file_list.append(os.path.join(frames_dir, file_name))
file_list = sorted(file_list)

Step 3: get bounding boxes of each frame.

In [5]:
detections = original_wrapper.main_wrapper(file_list)
vc.save(bbox_save_path, detections)


100%|##########| 381/381 [00:40<00:00,  9.52it/s]


The result format is like:
```
detect_results = {
    'path_to_frame_1': {
        'image_shape': (height, width, 3),
        'type': 'hand',
        'instances': [{'bbox': (x1, y1, x2, y2, conf}]
    },
    ...
}
```
Only the instance with the largest confidence will be kept, i.e. `len(instances) == 1`.